**Run code from the example.py** in https://github.com/wenyan4work/STKFMM/tree/master/Python

In [1]:
import numpy as np
import sys
from mpi4py import MPI
import PySTKFMM
import kernels as kr
import timer
import matplotlib.pyplot as plt

In [2]:
def calc_true_value(kernel_index, src_SL_coord, trg_coord, src_SL_value):
    epsilon_distance = 2e-4
    if kernel_index == PySTKFMM.KERNEL.PVel:
        trg_value  = kr.StokesSLPVel(src_SL_coord, trg_coord, src_SL_value, epsilon_distance = epsilon_distance)
        trg_value += kr.StokesDLPVel(src_DL_coord, trg_coord, src_DL_value, epsilon_distance = epsilon_distance)
    elif kernel_index == PySTKFMM.KERNEL.PVelGrad:
        trg_value  = kr.StokesSLPVelGrad(src_SL_coord, trg_coord, src_SL_value, epsilon_distance = epsilon_distance)
        trg_value += kr.StokesDLPVelGrad(src_DL_coord, trg_coord, src_DL_value, epsilon_distance = epsilon_distance)
    elif kernel_index == PySTKFMM.KERNEL.PVelLaplacian:
        trg_value  = kr.StokesSLPVelLaplacian(src_SL_coord, trg_coord, src_SL_value, epsilon_distance = epsilon_distance)
        trg_value += kr.StokesDLPVelLaplacian(src_DL_coord, trg_coord, src_DL_value, epsilon_distance = epsilon_distance)
    elif kernel_index == PySTKFMM.KERNEL.Traction:
        trg_value  = kr.StokesSLTraction(src_SL_coord, trg_coord, src_SL_value, epsilon_distance = epsilon_distance)
        trg_value += kr.StokesDLTraction(src_DL_coord, trg_coord, src_DL_value, epsilon_distance = epsilon_distance)
    elif kernel_index == PySTKFMM.KERNEL.LapPGrad:
        trg_value  = kr.LaplaceSLPGrad(src_SL_coord, trg_coord, src_SL_value, epsilon_distance = epsilon_distance)
        trg_value += kr.LaplaceDLPGrad(src_DL_coord, trg_coord, src_DL_value, epsilon_distance = epsilon_distance)
    else:
        trg_value = None
    return trg_value


    print('# Start')

# FMM parameters
mult_order = 10
max_pts = 128
pbc = PySTKFMM.PAXIS.NONE
kernels = [PySTKFMM.KERNEL.PVel, PySTKFMM.KERNEL.PVelGrad, PySTKFMM.KERNEL.PVelLaplacian, PySTKFMM.KERNEL.Traction, PySTKFMM.KERNEL.LapPGrad]
kernels_index = [PySTKFMM.KERNEL(k) for k in kernels]
verify = True

# Get MPI parameters
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Create sources and targets coordinates
nsrc_SL = 1000
nsrc_DL = 1000
ntrg = 1000
src_SL_coord = np.random.rand(nsrc_SL, 3)
src_DL_coord = np.random.rand(nsrc_DL, 3)
trg_coord = np.random.rand(ntrg, 3)
sys.stdout.flush()
comm.Barrier()

# Try STKFMM
for k, kernel in enumerate(kernels):
    # Create FMM
    print('\n\n==============================')
    timer.timer('create_fmm')
    myFMM = PySTKFMM.Stk3DFMM(mult_order, max_pts, pbc, kernels_index[k])
    timer.timer('create_fmm')
    timer.timer('show_active_kernels')
    myFMM.showActiveKernels()
    timer.timer('show_active_kernels')
    timer.timer('get_kernel_dimension')
    kdimSL, kdimDL, kdimTrg = myFMM.getKernelDimension(kernel)
    timer.timer('get_kernel_dimension')

    # Create sources and target values
    src_SL_value = np.random.randn(nsrc_SL, kdimSL) 
    src_DL_value = np.random.randn(nsrc_DL, kdimDL) 
    trg_value = np.zeros((ntrg, kdimTrg))
    print('kdimSL = ', kdimSL)
    print('kdimDL = ', kdimDL)
    print('kdimTrg = ', kdimTrg)

    # Set tree
    timer.timer('set_box')
    myFMM.setBox([0.0, 0.0, 0.0], 2)
    timer.timer('set_box')
    timer.timer('set_points')
    myFMM.setPoints(nsrc_SL, src_SL_coord, ntrg, trg_coord, nsrc_DL, src_DL_coord)
    timer.timer('set_points')
    timer.timer('setup_tree')
    myFMM.setupTree(kernel)
    timer.timer('setup_tree')

    # Evaluate FMM
    timer.timer('evaluate_fmm')
    myFMM.evaluateFMM(kernel, nsrc_SL, src_SL_value, ntrg, trg_value, nsrc_DL, src_DL_value)
    timer.timer('evaluate_fmm')

    # Clear FMM and evaluate again
    trg_value[:,:] = 0
    timer.timer('clear_fmm')
    myFMM.clearFMM(kernel)
    timer.timer('clear_fmm')
    timer.timer('evaluate_fmm')
    myFMM.evaluateFMM(kernel, nsrc_SL, src_SL_value, ntrg, trg_value, nsrc_DL, src_DL_value)
    timer.timer('evaluate_fmm')

    if verify:
        timer.timer('true_value')
        trg_value_true  = calc_true_value(kernels_index[k], src_SL_coord, trg_coord, src_SL_value)
        timer.timer('true_value')
        if trg_value_true is not None:
            diff = trg_value - trg_value_true
            print('relative L2 error = ', np.linalg.norm(diff) / np.linalg.norm(trg_value_true))
            print('Linf error        = ', np.linalg.norm(diff.flatten(), ord=np.inf))

comm.Barrier()
timer.timer(' ', print_all=True)
print('# End')



kdimSL =  4
kdimDL =  9
kdimTrg =  4
relative L2 error =  4.066312794266245e-10
Linf error        =  0.0006822492218816478


kdimSL =  4
kdimDL =  9
kdimTrg =  16
relative L2 error =  1.118463982605402e-10
Linf error        =  0.051843009641743265


kdimSL =  4
kdimDL =  9
kdimTrg =  7
relative L2 error =  2.2460632710133607e-09
Linf error        =  2.144615495344624


kdimSL =  4
kdimDL =  9
kdimTrg =  9
relative L2 error =  1.7142183763255178e-09
Linf error        =  0.003575384966097772


kdimSL =  1
kdimDL =  3
kdimTrg =  4
relative L2 error =  2.949191700260836e-09
Linf error        =  0.0017802201593895006


                      =  0
clear_fmm             =  0.0022287368774414062
create_fmm            =  19.636261701583862
evaluate_fmm          =  1.6786341667175293
get_kernel_dimension  =  1.8358230590820312e-05
set_box               =  0.00015974044799804688
set_points            =  0.0015192031860351562
setup_tree            =  0.3679368495941162
show_active_kernels   =  0.

**OK**